# Esercizio 6

Dato un file in formato GTF (Gene Transfer Format) che annota un set di geni sulla stessa genomica di riferimento, e il file della genomica di riferimento (*genomic reference*) in formato FASTA, produrre in output:

- per ognuno degli esoni annotati:
    - l'elenco degli identificatori dei trascritti che includono un dato esone insieme al gene di riferimento
    - l'elenco degli identificatori dei trascritti (insieme al gene di riferimento) per cui un dato esone è coperto completamente da CDS, specificando (per ogni trascritto) la porzione dell'esone coperta da CDS e la sua suddivisione in codoni

***

Parametri in input:

- nome del file in formato GTF
- nome del file della *genomic reference* in formato FASTA

***

Requisiti:

- deve essere definita una funzione `reverse_complement_in_case()` che prenda come argomento una sequenza nucleotidica e un valore di strand e ne restituisca il reverse&complement se lo strand è `-`, altrimenti restituisce la sequenza così com'è
- deve essere definita una funzione `codon_separating()` che prenda come argomento una sequenza nucleotidica e il frame in `{0, 1, 2}` e ne restituisca la suddivisione in codoni


**NOTA BENE**: gli attributi (coppie *nome-valore*) del nono campo del file GTF non devono essere pensati a ordine fisso all'interno del campo. Per estrarre quindi un attributo, non si può usare il metodo `split()`, ma si deve necessariamente usare un'espressione regolare.

***

Variabili di output:
- `exon_inclusion_dict`: dizionario di inclusione degli esoni: la chiave è la tupla *(start, end)* dell'esone e il valore è il set delle tuple *(transcript_id, gene_id)* che includono l'esone
- `exon_coverage_list`: lista degli esoni coperti interamente da coding sequence con la relativa suddivisione in codoni; ogni elemento è una tuple di cinque elementi: identificatore del trascritto, identificatore del gene, start dell'esone, end dell'esone, sudivisione in codoni (tenendo conto del valore di frame).   

***

### Note sul formato GTF

#### Esone incluso in trascritti diversi

Se un dato esone, inteso come intervallo `[start, end]` di posizioni sulla genomica di riferimento, è incluso in n trascritti di un gene, allora nel file GTF ci saranno n *record* di tipo `exon` con quei valori di start ed end. Gli identificatori di ognuno degli n trascritti che includono l'esone si trovano nel nono campo degli n *record*.

Ad esempio i *record* seguenti:

    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-014"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-002"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-003"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-001"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-024"; gene_id "ARHGAP4";
    ENm006 VEGA_Known	exon	64566	64757	.	-	.	transcript_id "U52112.4-011"; gene_id "ARHGAP4";
    
indicano che l'esone `[64566, 64757]` è incluso in sei trascritti del gene `ARHGAP4`.

#### Esone coperto da coding sequence

Se un dato esone `[start, end]`, incluso in un dato trascritto, è coperto (anche parzialmente) da CDS, allora ci sarà un *record* di tipo `CDS`, il cui nono campo contiene l'identificatore di tale trascritto, e il cui intervallo `[start_cds, end_cds]`, di inizio e fine sulla genomica di riferimento, coincide con l'intervallo `[start, end]` dell'esone, oppure ne occupa la parte finale (suffisso) o la parte iniziale (prefisso).

Ad esempio i due *record*:

    ENm006 VEGA_Known exon	70312	70440	.	-	.	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";

indicano che l'esone `[70312, 70440]` (incluso nel trascritto `U52112.4-005`del gene `ARHGAP4`) è coperto completamente dalla coding sequence del trascritto.

#### Suddivisione in codoni di una *feature* di tipo `CDS`

Dato un *record* di tipo `CDS`, la suddivisione in codoni della sequenza della *feature* sulla genomica di riferimento che esso rappresenta (attenzione: è una porzione della coding sequence del trascritto), deve tenere conto del valore del campo *frame* che specifica la posizione della prima base della sequenza all'interno del codone di appartenenza.

Ad esempio i tre *record* di tipo `CDS` seguenti:

    ENm006 VEGA_Known CDS	70312	70440	.	-	0	transcript_id "U52112.4-005"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	72521	72683	.	-	1	transcript_id "U52112.4-019"; gene_id "ARHGAP4";
    ENm006 VEGA_Known CDS	72761	72965	.	-	2	transcript_id "U52112.4-003"; gene_id "ARHGAP4";

rappresentano tre *features* di tipo `CDS` con i tre valori possibili del campo *frame*. 

Il valore 0 indica che la prima base della sequenza della *feature* CDS che esso rappresenta è la prima base di un codone (cioé le prime tre basi sono un codone completo).
Il valore 1 indica che la prima base è la seconda base di un codone, mentre 2 che la prima base è la terza base di un codone.

Le sequenze delle *features* rappresentate dai tre *record* sono rispettivamente:                        
 
    cggcaggccaagttcatggagcacaaactcaagtgcacaaaggcgcgcaacgagtacctgcttagcctggctagtgtcaacgctgctgtcagtaactactacctgcatgacgtcttggacctcatggac
    
    gaaggagccgtccctcctgtcgcccttgcactgctgggcggtgctgctgcagcacacgcggcagcagagccgggagagcgcggccctgagtgaggtgctggccgggcccctggcccagcgcctgagtcacattgcagaggacgtggggcgcctggtcaagaag
    
    agatgcgctggcagctgagcgagcagctgcgctgcctggagctgcagggcgagctgcggcgggagttgctgcaggagctggcagagttcatgcggcgccgcgctgaggtggagctggaatactcccggggcctggaaaagctggccgagcgcttctccagccgtggaggccgcctggggagcagccgggagcaccaaagcttccg

e le loro suddivisioni in codoni saranno dunque:

    cgg cag gcc aag ttc atg gag cac aaa ctc aag tgc aca aag gcg cgc aac gag tac ctg ctt agc ctg gct agt gtc aac gct gct gtc agt aac tac tac ctg cat gac gtc ttg gac ctc atg gac
    
    ga agg agc cgt ccc tcc tgt cgc cct tgc act gct ggg cgg tgc tgc tgc agc aca cgc ggc agc aga gcc ggg aga gcg cgg ccc tga gtg agg tgc tgg ccg ggc ccc tgg ccc agc gcc tga gtc aca ttg cag agg acg tgg ggc gcc tgg tca aga ag
    
    a gat gcg ctg gca gct gag cga gca gct gcg ctg cct gga gct gca ggg cga gct gcg gcg gga gtt gct gca gga gct ggc aga gtt cat gcg gcg ccg cgc tga ggt gga gct gga ata ctc ccg ggg cct gga aaa gct ggc cga gcg ctt ctc cag ccg tgg agg ccg cct ggg gag cag ccg gga gca cca aag ctt ccg